In [4]:
import pandas as pd
import numpy as np
import requests
import json
import time
import sys
from pymongo import MongoClient

#Connessione al DB
client = MongoClient(host='localhost', port=27018)
db = client.kiva_DB 

# VOLUME + VARIETA'
#### DOWNLOAD DEI LOANS MEDIANTE API DI KIVA.ORG, ARRICCHIMENTO CON I DETTAGLI, AGGIUNTA DELLE INFORMAZIONI SUGLI INDICATORI ECONOMICI (gdp, gni, gni-metodo atlas e gni per capita) E SALVATAGGIO SU MONGODB

In [5]:
#URL e parametri per contattare le API, + dati dalla World Bank per la varietà
url = 'https://api.kivaws.org/v1/loans/search.json'
params = {
        'per_page' : 100,
        'sort_by' : 'oldest',
        'ids_only': 'true'
            }

###################SOSTITUISCI FOLDER CON IL PERCORSO DELLA CARTELLA DOVE HAI MESSO I CSV DALLA WORLD BANK###################
##############################OCCHIO A LASCIARE '/{}.csv' ALLA FINE DEL PERCORSO DELLA CARTELLA##############################
folder = 'C:/Users/pablo/Desktop/DS - Primo anno Primo semestre/Data_Management/Progetto/{}.csv'


indicators = {'GDP' : pd.read_csv(folder.format('GDP')).set_index('Unnamed: 0'),
                 'GNI': pd.read_csv(folder.format('GNI')).set_index('Unnamed: 0'),
                 'GNI_atlas': pd.read_csv(folder.format('GNI_atlas')).set_index('Unnamed: 0'),
                 'GNI_pc': pd.read_csv(folder.format('GNI_pc')).set_index('Unnamed: 0')}

df_GDP = pd.melt(indicators['GDP'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GDP'}).reset_index(drop=True)
df_GNI = pd.melt(indicators['GNI'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GNI'}).reset_index(drop=True)
df_GNI_atlas = pd.melt(indicators['GNI_atlas'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GNI_atlas'}).reset_index(drop=True)
df_GNI_pc = pd.melt(indicators['GNI_pc'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GNI_pc'}).reset_index(drop=True)


In [2]:
def get_economic_indicator(indicator,country, year, indicators):
    
   
    df = indicators[indicator]
    if(str(year) in df.columns):
        if(country in df.index): 
            return df.loc[country].to_dict()[str(year)]
        else:
            return np.nan
    
def get_string_of_IDs(nums):
    IDs = []
    for i in nums:
        IDs.append(str(i))
    return ','.join(IDs)

In [4]:
#Per poter effettuare le richieste in più tempi, è opportuno poter fermare lo script e farlo ripartire.
#Quando lo script parte, chiede al DB quanti loans contenga e da questa cifra ricava il numero di pagina iniziale.

#Ddue problemi: la prima pagina per le API non è indicata come page=0, come sarebbe comodo, ma semplicemente manca
#il parametro page nella richiesta, quindi bisogna fare una richiesta apposta per la prima pagina.
count = db.loans.count()
print('Initial count: ',count)
start = int((count - count%100)/100) + 1
if(count == 0):
    start = 0
elif(count%100 == 0):
    start = int(count/100) + 1
    
#Il numero di pagina finale invece viene chiesto direttamente alle API
res_end = requests.get(url = url, params = params).json()
end = res_end['paging']['pages']

#Può capitare che una richiesta non vada a buon fine perchè le API hanno un limite di richieste (nel
#qual caso ritento dopo 1 minuto), oppure per altri motivi (ad es risorsa mancante), nel qual caso occorre 
#intervento umano.
#Per distinguere tra i due casi setto un contatore che mi indichi quante volte la richiesta è fallita; quando
#una richiesta fallisce la prima volta allora ritento dopo 1 minuto, ma se fallisce per 5 volte consecutive,
#assumo che ci sia un problema più serio e chiudo lo script.
fails_counter=0

#####################################################   Si comincia.   ####################################################
for page in range(start,end):
    
     if(res_end['paging']['total'] - db.loans.count() < 100):  
#Finito di scaricare e salvare, chiudo la connessione con il DB e termino il programma
        db.client.close()
        sys.exit()

#Per stimare quanto tempo occorra ad ottenere ciascuna pagina, salvo innanzitutto il tempo attuale.
    start_time = time.time()
#Setto la pagina attuale da richiedere alla API. Appunto, se mi serve la prima pagina allora mando la richiesta senza
#il parametro 'page', altrimenti devo settarlo qui.
    if(page != 0):
        params['page']= str(page)
        
   #ora invio la richiesta per i soli ID dei loans della pagina che sto chiedendo e salvo la risposta, chiamandola r_ID
    r_ID = requests.get(url = url, params = params)
    
#Questo if gestisce il fallimento della richiesta come descritto sopra
    if(r_ID.status_code != 200):
        fails_counter +=1
        
        if(fails_counter >5):
            print('Too many fails, shutting down at page ', page, ' with code ', str(r_ID.status_code))
            sys.exit()
        else:
            print('Request was denied at page ', page, ' with code ',str(r_ID.status_code), '. I\'ll try again in a minute. Fail counter: ', fails_counter)
            time.sleep(62)
            page = page -1
            
#Se si arriva a questo punto allora la richiesta è andata bene, quindi recupero gli ID, 
#invio la richiesta per avere i dettagli e li aggiungo a ciascun loan
    else: 

#Innanzitutto resetto il contatore dei fails
        fails_counter=0
    
#Ho gli IDs, li uso per scaricare i loans.
        IDs_string = get_string_of_IDs(r_ID.json()['loans'])
        
#Ora ho la stringa degli IDs, posso lanciare la richiesta per i loans veri e propri
        url_d = 'https://api.kivaws.org/v1/loans/{}.json'.format(IDs_string)
        r_d = requests.get(url_d)

#Come prima, potrei ricevere una risposta non valida (risorsa mancante ecc), ma in tal caso semplicemente
#non salvo quel blocco di loans, perchè sarebbe inutile
        if(r_d.status_code == 200):
            rj = r_d.json()['loans']
            
#Aggiungo la parte di varietà: ad ogni loan vengono aggiunti gli opportuni campi con GDP, GNI ecc
            for loan in rj:
                loan['location']['GDP']= get_economic_indicator('GDP',loan['location']['country'], loan['posted_date'][0:4], indicators)
                loan['location']['GNI']= get_economic_indicator('GNI',loan['location']['country'], loan['posted_date'][0:4], indicators)
                loan['location']['GNI_atlas']= get_economic_indicator('GNI_atlas',loan['location']['country'], loan['posted_date'][0:4], indicators)
                loan['location']['GNI_pc']= get_economic_indicator('GNI_pc',loan['location']['country'], loan['posted_date'][0:4],indicators)

#Ora finalmente il blocco di loans è completo di dettagli e pronto ad essere mandato al DB
    db.loans.insert_many(rj)
    
#Tanto per avere un po' di controllo sull'esecuzione, ad ogni ciclo stampo il numero di loans ricevuti ed il tempo 
#occorso a compiere tutte le operazioni
    time_elapsed = time.time() - start_time
    print('Loans: ', db.loans.count(), ', time elapsed: ', time_elapsed, end = '\r')
   

Initial count:  1255336


SystemExit: 

C:\Users\pablo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
